In [ ]:
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from dotenv import load_dotenv
import os

load_dotenv()

tenant_id = os.environ.get("TENANT_ID") 
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
account_name = os.environ.get("PURVIEW_ACCOUNT")

auth = ServicePrincipalAuthentication(
    tenant_id = tenant_id, 
    client_id = client_id, 
    client_secret = client_secret
)

# Create a client to connect to your service.
client = PurviewClient(
    account_name = account_name,
    authentication = auth
)

In [ ]:
client.get_all_typedefs()

In [ ]:
client.endpoint_url

In [ ]:
client.get_entity(qualifiedName="https://purviewatlaspoc.dfs.core.windows.net/datalake/extended_vaccine_data/{SparkPartitions}", typeName="azure_datalake_gen2_resource_set")

In [ ]:
client.get_relationship(guid='a4021762-cd09-4f75-bf80-ea727e1e13b2')